# Economic Experiment

Add description of the experiment here

## Import Statements

In [1]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore, Likelihood, ProbabilityDifference, ProbabilityPointer, FavouriteProbability
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.evaluators.base_evaluators import BettingActivity
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError

from dfg_rating.logic.controller import Controller
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import itertools as it

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Experiment configuration

Two main things: The controller is the helper that includes the functions to load and save networks. A configuration object with the parameters of the experiment, more cells can be created with different configurations.

In [2]:
main_controller = Controller()

In [3]:
experiment1 = dict(
    test_name="Experiment1", create_data=True,
    bookmaker_error=0.0, bookmaker_margin=0.10,
    betting_bankrole=100,
    rating_error_loc=0, rating_error_scale=50,
    bookmaker_rating_error_loc=0, bookmaker_rating_error_scale=25,
    number_of_leagues=40,
    number_of_teams=20, number_of_seasons=10,
    true_forecast=dict(
        coefficients=[-0.9,0.3],
        beta=0.006
    ),
    bookmaker_forecast=dict(
        coefficients=[-1.2, 0.0],
        beta=0.006
    ),
    in_sample_maximum=5,
    number_of_splits=20
)

In [4]:
experiment2 = dict(
    test_name="Experiment2", create_data=True,
    bookmaker_error=0.0, bookmaker_margin=0.10,
    betting_bankrole=100,
    rating_error_loc=0, rating_error_scale=50,
    bookmaker_rating_error_loc=0, bookmaker_rating_error_scale=25,
    number_of_leagues=40,
    number_of_teams=20, number_of_seasons=10,
    true_forecast=dict(
        coefficients=[-0.9,0.3],
        beta=0.006
    ),
    bookmaker_forecast=dict(
        coefficients=[-0.8, 0.2],
        beta=0.006
    ),
    in_sample_maximum=5,
    number_of_splits=20
)

In [5]:
experiment3 = dict(
    test_name="Experiment3", create_data=True,
    bookmaker_error=0.0, bookmaker_margin=0.10,
    betting_bankrole=100,
    rating_error_loc=0, rating_error_scale=50,
    bookmaker_rating_error_loc=0, bookmaker_rating_error_scale=25,
    number_of_leagues=40,
    number_of_teams=20, number_of_seasons=10,
    true_forecast=dict(
        coefficients=[-0.9,0.3],
        beta=0.006
    ),
    bookmaker_forecast=dict(
        coefficients=[-0.8, 0.4],
        beta=0.004
    ),
    in_sample_maximum=5,
    number_of_splits=20
)

### !!Select the right config

In [6]:
# The desired config is stored at config
config = add_config_name_here

## External actors

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [7]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=config["bookmaker_error"], scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=config["bookmaker_margin"])
)

In [8]:
betting = FixedBetting(config["betting_bankrole"])

We also create an error rating that we are going to use for the calculated forecasts

In [9]:
rating_error = RatingFunctionError(error='normal', loc=config["rating_error_loc"], scale=config["rating_error_scale"])
rating_error_bookmaker = RatingFunctionError(error='normal', loc=config["bookmaker_rating_error_loc"], scale=config["bookmaker_rating_error_scale"])

## Generation of networks

Configure the number of leagues for the experiment:

In [10]:
numberLeagues = config["number_of_leagues"]

For every league, we generate the schedule of matches and true dimensions and we add odds for each game.

In [11]:
networks = []

In [12]:
gen_start_time = time.time()
for league in range(numberLeagues):
    if config["create_data"]:
        network: BaseNetwork = factory.new_network(
            'multiple-round-robin',
            teams=config["number_of_teams"],
            days_between_rounds=7,
            seasons=config["number_of_seasons"],
            league_teams=config["number_of_teams"],
            league_promotion=0,
            create=True,
            true_forecast=LogFunctionForecast(
                outcomes=['home', 'draw', 'away'], 
                coefficients = config["true_forecast"]["coefficients"], 
                beta_parameter=config["true_forecast"]["beta"]
            ),
            true_rating=ControlledTrendRating(
                starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
                delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
                trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
                season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)
            )
        )
        #adding biased bookmaker forecast to the network
        network.add_forecast(
            forecast=LogFunctionForecast(
                outcomes=['home', 'draw', 'away'], 
                coefficients = config["bookmaker_forecast"]["coefficients"], 
                beta_parameter=config["bookmaker_forecast"]["beta"], 
                home_team_error=rating_error_bookmaker, 
                away_team_error=rating_error_bookmaker
            ),
            forecast_name='bookmaker_forecast',
            base_ranking='true_rating'
        )
        #adding odds    
        network.add_odds(
            bookmaker_name="bm",
            bookmaker=bookmaker,
            base_forecast='bookmaker_forecast'
        )
    else:
        print("Loading network data")
        main_controller.load_network_from_sql(
            network_name=f"{config['test_name']}_network_{league}",
            new_network_name=f"{config['test_name']}_network_{league}"
        )
        network: BaseNetwork = main_controller.networks[f"{config['test_name']}_network_{league}"]
    networks.append(network)
print(f"{numberLeagues} leagues added in {float(time.time() - gen_start_time)} seconds.")

40 leagues added in 41.50503420829773 seconds.


# Run

We iterate over the grid of possible parameters for the experiment and aggregate the final results.

### Functions

In [13]:
def aggregate_measures(list_of_matches, prefix='', addBettingMetrics=True):
    if addBettingMetrics:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches]),
            prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
            prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches]),
            prefix+'_betting_activity': sum([m[3]['metrics']['betting_activity']['qty'] for m in list_of_matches])
        }
    else:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches])
        }

Getting first the values for the true model and the bookmaker model. Remove some of the if they are not needed and if the IS/OOS split is not required.

In [14]:
entire_model_values = {}
for forecast_pointer in ['true_forecast', 'bookmaker_forecast']:
    all_matches =  []
    is_matches =  []
    oos_matches =  []
    for network in tqdm(networks):
        rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer, probability_index=1)
        fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        network.add_evaluation([
            (rps, 'rps'),
            (likelihood, 'likelihood'),
            (difference, 'prob_diff'),
            (draw_probability, 'draw_prob'),
            (fav_probability, 'fav_prob')
        ])
        all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
        is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 6]
        oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 5]

    result_all = aggregate_measures(all_matches, f'all_{forecast_pointer}', False)
    result_is = aggregate_measures(is_matches, f'is_{forecast_pointer}', False)
    result_oos = aggregate_measures(oos_matches, f'oos_{forecast_pointer}', False)
    entire_model_values = {
        **entire_model_values,
        **result_is, **result_oos, **result_all
    }

100%|██████████| 40/40 [00:04<00:00,  8.32it/s]


Cross validation spits and results of each permutation

In [15]:
def cross_validate_betting_returns(matches_array, n_splits, model_hint):
    kf = np.array_split(matches_array, n_splits)
    combinations = [(in_s, out_s) for in_s, out_s in it.permutations(range(len(kf)), r=2)]
    result = []
    for in_sample_index, out_sample_index in combinations:
        result_is = aggregate_measures(kf[in_sample_index], "is")
        result_oos = aggregate_measures(kf[out_sample_index], "oos")
        result.append({
            "permutation": f"{in_sample_index} x {out_sample_index}",
            "model_hint": model_hint,
            **result_is,
            **result_oos
        })
    return result

### Initialization of results and cross validation output

In [16]:
result_list = []
cv_result = []

### Main loop

In [17]:
experiment_start_time = time.time()
for c0 in np.arange(-1.20, -0.60, 0.1):
    for c1 in np.arange(0.0, 0.60, 0.1):
        for beta in np.arange(0.002, 0.010, 0.002):
            print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
            forecast_pointer = f"player_forecast_{c0:.2f}_{c1:.2f}_{beta:.3f}"
            cell_start_time = time.time()
            all_matches =  []
            is_matches =  []
            oos_matches =  []
            
            for network_number, network in enumerate(networks):
                if config["create_data"]:
                    network.add_forecast(
                        forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
                        forecast_name=forecast_pointer,
                        base_ranking='true_rating'
                    )
                network.add_bets(
                    bettor_name='b',
                    bookmaker='bm',
                    betting=betting,
                    base_forecast=forecast_pointer
                )
                
                rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
                betting_activity = BettingActivity(outcomes=['home', 'draw', 'away'], player_name = 'b')
                likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer, probability_index=1)
                fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)

                network.add_evaluation([
                    (rps, 'rps'),
                    (betting_returns, 'betting_returns'),
                    (betting_activity, 'betting_activity'),
                    (likelihood, 'likelihood'),
                    (difference, 'prob_diff'),
                    (draw_probability, 'draw_prob'),
                    (fav_probability, 'fav_prob')
                ])
                    
                all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
                is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] <= config["in_sample_maximum"]]
                oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > config["in_sample_maximum"]]
            
            cv_result += cross_validate_betting_returns(np.array(all_matches), config["number_of_splits"], forecast_pointer)
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all, **entire_model_values
            }
            result_list.append(result)
            print(f"Finished in {float(time.time() - cell_start_time)} seconds")

print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")

df = pd.DataFrame(result_list)
df.to_excel(f"Results_{config['test_name']}.xlsx")

if config["create_data"]:
    print("Saving data with the all the forecasts added")
    for network_number, network in tqdm(enumerate(networks)):
        main_controller.networks[f"{config['test_name']}_network_{network_number}"] = network
        main_controller.save_network(f"{config['test_name']}_network_{network_number}")
            

Variables: c0: -1.2, c1: 0.0 and beta: 0.002
Finished in 42.601314544677734 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.004
Finished in 43.71205449104309 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.006
Finished in 42.93880319595337 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.008
Finished in 44.22097039222717 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.002
Finished in 43.608847856521606 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.004
Finished in 44.04694652557373 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.006
Finished in 45.10042333602905 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.008
Finished in 45.073723554611206 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.002
Finished in 43.530656814575195 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.004
Finished in 44.54267120361328 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.006
Finished in 44.6839394569397 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.008
Finished in 43.76976442337036 seconds
V

0it [00:00, ?it/s]

('PostgreSQL 12.5, compiled by Visual C++ build 1914, 64-bit',)
dfg_rating database exists. Loading database
networks 1
INSERT INTO networks(network_name,network_type,seasons,days_between_rounds,rounds) VALUES %s ON CONFLICT DO NOTHING
matches 3800
INSERT INTO matches(network_name,node1,node2,season,round,day,winner,match_id) VALUES %s ON CONFLICT DO NOTHING
forecasts 554800
INSERT INTO forecasts(forecast_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
odds 3800
INSERT INTO odds(network_name,bookmaker_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
bets 3800
INSERT INTO bets(bettor_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
metrics 26600
INSERT INTO metrics(metric_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
ratings 15800
INSERT INTO ratings(rating_name,node_id,node_name,network_name,season,rating_number,value,trend,starting_point) VALUES %s ON CONFLICT DO NOTHING


1it [00:52, 52.63s/it]

networks 1
INSERT INTO networks(network_name,network_type,seasons,days_between_rounds,rounds) VALUES %s ON CONFLICT DO NOTHING
matches 3800
INSERT INTO matches(network_name,node1,node2,season,round,day,winner,match_id) VALUES %s ON CONFLICT DO NOTHING
forecasts 554800
INSERT INTO forecasts(forecast_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
odds 3800
INSERT INTO odds(network_name,bookmaker_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
bets 3800
INSERT INTO bets(bettor_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
metrics 26600
INSERT INTO metrics(metric_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
ratings 15800
INSERT INTO ratings(rating_name,node_id,node_name,network_name,season,rating_number,value,trend,starting_point) VALUES %s ON CONFLICT DO NOTHING


...

38it [41:44, 53.19s/it]

networks 1
INSERT INTO networks(network_name,network_type,seasons,days_between_rounds,rounds) VALUES %s ON CONFLICT DO NOTHING
matches 3800
INSERT INTO matches(network_name,node1,node2,season,round,day,winner,match_id) VALUES %s ON CONFLICT DO NOTHING
forecasts 554800
INSERT INTO forecasts(forecast_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
odds 3800
INSERT INTO odds(network_name,bookmaker_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
bets 3800
INSERT INTO bets(bettor_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
metrics 26600
INSERT INTO metrics(metric_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
ratings 15800
INSERT INTO ratings(rating_name,node_id,node_name,network_name,season,rating_number,value,trend,starting_point) VALUES %s ON CONFLICT DO NOTHING


39it [42:36, 52.78s/it]

networks 1
INSERT INTO networks(network_name,network_type,seasons,days_between_rounds,rounds) VALUES %s ON CONFLICT DO NOTHING
matches 3800
INSERT INTO matches(network_name,node1,node2,season,round,day,winner,match_id) VALUES %s ON CONFLICT DO NOTHING
forecasts 554800
INSERT INTO forecasts(forecast_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
odds 3800
INSERT INTO odds(network_name,bookmaker_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
bets 3800
INSERT INTO bets(bettor_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
metrics 26600
INSERT INTO metrics(metric_name,network_name,match_id,attributes) VALUES %s ON CONFLICT DO NOTHING
ratings 15800
INSERT INTO ratings(rating_name,node_id,node_name,network_name,season,rating_number,value,trend,starting_point) VALUES %s ON CONFLICT DO NOTHING


40it [43:29, 65.24s/it]


### Process and store cross-validation results

In [18]:
cv_df = pd.DataFrame(cv_result)

cv_df[cv_df['permutation'] == "0 x 1"].sort_values(by='is_rps').head(144)

# Optimizing for RPS
by_rps_df = cv_df.loc[cv_df.groupby('permutation').is_rps.idxmin(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')
# Optimizing for likelihood
by_likelihood_df = cv_df.loc[cv_df.groupby('permutation').is_likelihood.idxmax(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')
# Optimizing for betting
by_betting_df = cv_df.loc[cv_df.groupby('permutation').is_betting_returns.idxmax(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')

cv_study_df = by_rps_df.join(
    by_likelihood_df.join(
        by_betting_df,
        lsuffix='',
        rsuffix='_betting',
    ),
    lsuffix='_rps',
    rsuffix='_likelihood',
)

cv_study_df.to_excel(f"Permutations_{config['test_name']}.xlsx")